### Reads in data frame (from pickle)
### Subsets data frame (one list per month)
### Pulls lyrics from mxm database given input dates and max rank (i.e. top 20, or top 50, or top 100)

### This all should work so long as you have the csv files (from google drive) in the right place with the right name, and you have the Billboard100DF in your working directory. All you should have to change is the first line (put your mxm_key), and then in the cell that starts #for y in xrange(2007,2016): (for finishing the 2000s), you should edit that top lie to reflect the dates you want to pull. You can also change max_rank to something other than 20 if you want to get lyrics for more or fewer of the list.

In [13]:
# IMPORTANT: edit this cell with your API info for billboard and spotify
#mxm_key = "f8a009fe01e53de1e8a1388e60812ef5" #swuerth
#mxm_key = "76c5a287723780d7e5a64d298ab3c382" #swuerth22
#mxm_key = "ad4dd59d0bb77715f1d6db569cbae1a8" #claire
#mxm_key = '014aeed4569b07ec299c9a9271bb0a6c' #athena
mxm_key = 'f1b96add83c447f65b20ddb7d9fb4356' #elena

In [2]:
# Import everything we'll need in this notebook here
import billboard
from billboard import ChartData
import pdb 
import numpy as np 
import pandas as pd

# for lyrics stuff (mxm)
from musixmatch.track import TracksCollection
from musixmatch.lyrics import Lyrics

# for fuzzy matching artist names (when matching hot100 song to lyrics in mxm)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# for joining list of strings
import string


# for ascii / utf8 errors, run this
import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
print 'testing!'

import glob

import datetime

import csv


testing!


In [3]:
#before we read in the dataframe, let's get a list of dates included in the billboard dataset
#note this will only work if you have all the csv files that steph posted to google drive

file_list = glob.glob("./hot100/hot100*csv") #or wherever your csvs are stored
print 'num lists = ', len(file_list)

#
all_dates = [] #here we will store all the dates
date_ind_start = 16
date_ind_end = 26
for f in file_list:
    all_dates.append(f[date_ind_start:date_ind_end])
print 'first 4 dates are : ', all_dates[0:4]

# make another list 'monthly_dates' that only stores every 4th date from all_dates
monthly_dates = all_dates[0::4]
print 'subsetting to monthly gives ', len(monthly_dates), ' lists'
print 'first 4 dates of monthly subset are : ', monthly_dates[0:4]

#save monthly datelist to csv:
myfile = open('monthly_date_list.csv', 'wb')
wr = csv.writer(myfile)
wr.writerow(monthly_dates)

num lists =  3023
first 4 dates are :  ['1958-08-09', '1958-08-16', '1958-08-23', '1958-08-30']
subsetting to monthly gives  756  lists
first 4 dates of monthly subset are :  ['1958-08-09', '1958-09-06', '1958-10-04', '1958-11-01']


In [4]:
#read df from pickle file
df = pd.read_pickle('./Billboard100DF')

In [5]:
# we'll work with only month subset of full billboard df
## let's subset to monthly data using our monthly date list.
r = len(df)
print 'rows in full dataframe = ', r
# replace 'df' with only monthly lists
df = df[df['date'].isin(monthly_dates)].reindex()
print 'rows in monthly subset = ', len(df)
print 'ratio = ', len(df)/float(r)
df.head()

rows in full dataframe =  302142
rows in monthly subset =  75563
ratio =  0.250091016807


,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09


In [6]:
### Edit DataFrame such that rank, LostPos,PeakPos,Rank,Weeks are all integers ?
### Add numeric date fields

df['Weeks'] = pd.to_numeric(df['Weeks'])
df['Rank'] = pd.to_numeric(df['Rank'])
df['date'] = pd.to_datetime(df['date'])
# now that rank and date are in numeric form, we can subset with > & <
# for example
print '# songs in top 20 (all dates): ',len(df[df['Rank']<20])
print 'displaying just the parts from June 2016 onwards:'
df[df['date'] < datetime.date(year=1960,month=6,day=1)]

# songs in top 20 (all dates):  14355
displaying just the parts from June 2016 onwards:


,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09
5,Duane Eddy His Twangy Guitar And The Rebels,Re-Entry,0,6,6,6gIFUpY85r9b4pCDIIkJDF,https://embed.spotify.com/?uri=spotify:track:6...,Rebel-'Rouser,8,1958-08-09
6,The Coasters,Re-Entry,0,7,7,3h5g8M5cWCPniE29PHhv7M,https://embed.spotify.com/?uri=spotify:track:3...,Yakety Yak,7,1958-08-09
7,Jack Scott,Re-Entry,0,3,8,2urRqmAFhjZKo8Z6sEGzEv,https://embed.spotify.com/?uri=spotify:track:2...,My True Love,14,1958-08-09
8,The Johnny Otis Show,Re-Entry,0,9,9,5gjeRYKyOH23oNNb59WjJq,https://embed.spotify.com/?uri=spotify:track:5...,Willie And The Hand Jive,10,1958-08-09
9,Peggy Lee,Re-Entry,0,8,10,2CeqxyOZEyiL6pTDYZ9gPH,https://embed.spotify.com/?uri=spotify:track:2...,Fever,12,1958-08-09


In [8]:
# make function that takes a list of artists (from mxm), and selects the best match to the artist from hot100
def find_artist_match(name_to_match, artist_list):
    index = 0 #index of the artist we're scoring
    r = 0 #here i'll store the highest similarity score
    best_match = 'none'
    best_ind = -1 #initialize
    for a in artist_list:
        rnew = fuzz.ratio(name_to_match, a)
        #print 'similarity between ', name_to_match, ' and ', a, ' : ', rnew
        if rnew > r: 
            best_match = a
            best_ind = index
            r = rnew
        index = index+1

    print 'Best Match = ', best_match
    assert(best_match==artist_list[best_ind])
    print 'Score = ',r
    return {'ind':best_ind, 'score':r} 


In [8]:
#want 2007 to present day
#for y in xrange(2007,2016)
for y in xrange(1980,1990):
    print y
    

1980
1981
1982
1983
1984
1985
1986
1987
1988
1989


## Here is where you will want to start getting data. 
### Make sure your mxm key is stored in mxm_key at the top of this notebook.
### In the block below, change the years in xrange() to bracket the dates you want to get top 20 lists from
### As of 6pm Thursday, Jully 14, Stephanie has done this for years 2000-2015 and for 1990-1998.
### In the 1990's we still need './Lyrics_1999-01-01-2000-01-01MaxRank20' to be made, and then we need older decades 1950s, 60s, 70s, 80s.


In [9]:
# in the 1970s, ran into error Desire by Neil Diamond..
# df[df['Artist']=='Neil Diamond']
df.head()

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09


In [13]:
len(df)

75563

In [22]:
# prior to july 18 2016, the match between the billboard list and the 
# lyrics data was made by exact match on track name followed by fuzzy matching artist name
# it will be better if we first match on spotify id, then do the track+artist matching
apikey = mxm_key
##Stevie Wonder	Go Home
artist_hot = 'Stevie Wonder'
trackname_hot = 'Go Home'
print artist_hot, trackname_hot
collection = TracksCollection.fromSearch(q_artist=artist_hot, apikey = apikey)
#collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)

Stevie Wonder Go Home


In [28]:
df.head()

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09


In [24]:
cols = ['lyrics_id', 'has_lyrics', 'artist_name', 'track_spotify_id','track_id','track_name']
subset = pd.DataFrame.from_dict(collection)[cols]
subset

,lyrics_id,has_lyrics,artist_name,track_spotify_id,track_id,track_name
0,0,0,Wonder Stevie,,32607330,Seasons of Love (Instrumental Version)
1,13464979,1,Stevie Wonder,0dEAEf91tqOA3rm1YGS6uI,32792045,Higher Ground
2,145047,1,Stevie Wonder,3zeWA0JrlILWDSigKYqO3H,1869376,Stranger on the Shore of Love
3,4949404,1,Stevie Wonder,2jEikvInB6SoZtJe13QdtN,44242728,Heaven Is 10 Zillion Light Years Away
4,1796302,1,Stevie Wonder,10HB7aFgb9OeiP31NdUpaH,1500800,Jungle Fever
5,8357296,1,Stevie Wonder,,8897593,Living for the City / A Man Gotta Do
6,2912319,1,Stevie Wonder,1WeOWaMQRxuD9CsAL8KGgk,80852409,In Your Corner
7,1796240,1,Stevie Wonder,2bkULcY9tggS4COibw6YMA,18984502,From the Bottom of My Heart
8,1796253,1,Stevie Wonder,018ABzRSJ9dmfTmxKn4WpV,35498325,Hallelujah (I Love Her So)
9,4948560,1,Stevie Wonder,,1780470,My Love


In [21]:
tracks = subset['track_name'].values
result = find_artist_match(trackname_hot, tracks)
ind = result['ind']
score = result['score']
print 'matched track name = ', tracks[ind]

Best Match =  Bang Bang (My Baby Shot Me)
Score =  92
matched track name =  Bang Bang (My Baby Shot Me)


In [34]:
start_date = datetime.date(year=y,month=1,day=1)
end_date = datetime.date(year=y+1,month=1,day=1)
df[(df['date'] > start_date) & (df['date']< end_date)]

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date


In [52]:
# #for y in xrange(2007,2016): (for finishing the 2000s)
# for y in xrange(1958,1960): #get the 70s!

#     ## Initialize data frames and lists:
#     apikey = mxm_key

#     #header for info to be stored in df_mxm_info
#     cols = ['lyrics_id', 'has_lyrics', 'artist_name', 'track_spotify_id','track_id','track_name']

#     #header for info in df_mxm_lyrics
#     lyrics_df_cols = ['lyrics_copyright', 'lyrics_body','publisher_list', 'can_edit', 'writer_list', \
#                       'instrumental','verified','html_tracking_url','lyrics_language_description', \
#                       'locked','restricted','explicit','updated_time','lyrics_id','script_tracking_url', \
#                       'action_requested','pixel_tracking_url','lyrics_language']

#     # initialize dfs and lists :
#     #if 'df_mxm_info' not in locals():
#     df_mxm_info = pd.DataFrame(columns = cols) #dataframe for some song info (from collections object)
#     #if 'df_mxm_lyrics' not in locals():
#     df_mxm_lyrics = pd.DataFrame(columns = lyrics_df_cols) #dataframe for lyrics and related attributes
#     #if 'lyrics_list' not in locals():
#     lyrics_list = [] #append lyric info to a list for each song, later we'll put it in a dataframe!
#     #if 'hot_artists_no_lyric' not in locals():
#     hot_artists_no_lyrics = [] #when we don't find lyrics, store artistname here
#     #if 'hot_tracknames_no_lyrics' not in locals():
#     hot_tracknames_no_lyrics = [] #when we don't find lyrics, store trackname here
#     print 'df_mxm_info has ', len(df_mxm_info), ' rows' 
#     print 'df_mxm_lyrics has ', len(df_mxm_info), ' rows' 
#     print 'lyrics_list has length ', len(df_mxm_info)

#     #subset df_monthly given start_date, end_date, and max_rank:
#     start_date = datetime.date(year=y,month=1,day=1)
#     end_date = datetime.date(year=y+1,month=1,day=1)
#     max_rank = 20 #just top N songs

#     df_reduced = df[(df['date'] > start_date) & (df['date']< end_date) & (df['Rank']<=max_rank)]

#     print ' number songs in df_reduced = ', len(df_reduced)

#     #name for saving df
#     name2save = './new/Lyrics_'+str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)
#     print 'will save df as : ', name2save

#     #now get the lyrics
#     for i in xrange(len(df_reduced)):
#         trackname_hot = df_reduced['Title'].iloc[i]
#         #fix unicode error neil diamond desire song...
#         if trackname_hot=='Desir\xc3\x83\xc2\x83\xc3\x82\xc2\xa9e':
#             trackname_hot = 'Desire'
#         #fix for san jose
#         if trackname_hot=='Do You Know The Way To San Jos\xc3\x83\xc2\x83\xc3\x82\xc2\xa9':
#             trackname_hot = 'Do You Know The Way To San Jose'
#         if trackname_hot=='Nel Blu Dipinto Di Blu (Volar\xc3\x83\xc2\x83\xc3\x82\xc2\xa9)':
#             trackname_hot = 'Nel Blu Dipinto Di Blu Volar'
#         artist_hot = df_reduced['Artist'].iloc[i]
#         spotify_hot = df_reduced['SpotifyID'].iloc[i]
#         print 'processing ', trackname_hot, ' by ', artist_hot

#         #ask mxm for info about this track (search on trackname first)
#         collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)
#         if len(collection)==0:
#             print('No lyrics found')
#             hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#             hot_tracknames_no_lyrics.append(trackname_hot) 
#             continue # skip this row of df_reduced
#         #store collection in dataframe called 'subset'
#         subset = pd.DataFrame.from_dict(collection)[cols]
#         #filter for only songs with lyrics:
#         subset = subset[subset['has_lyrics']==1]
#         #move on if no matches found...
#         if len(subset)==0:
#             print ' No lyrics found ...'
#             hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#             hot_tracknames_no_lyrics.append(trackname_hot) 
#             continue # skip this row of df_reduced -- in future we should try other searches when this initial search fails
#         artists = subset['artist_name'].values
#         tracks = subset['track_name'].values
#         sids = subset['track_spotify_id'].values
#         track_id = subset['track_id'].values
#         print 'There are ', len(artists), ' artists with this song title'
#         # fuzzy match the artist from hot100 to all artists matching track name
#         try:
#             result = find_artist_match(artist_hot, artists)
#         except AssertionError:
#             continue
#         ind = result['ind']
#         score = result['score']
#         if score >= 75 or spotify_hot==sids[ind]:
#             print 'matched artist name = ', artists[ind]
#             # select the best track id (best match to artist name)
#             track_id_selected = track_id[ind]
#             # now get the lyrics, but only if the 'track_id' is not already in the info df
#             if track_id_selected in df_mxm_info.track_id.values:
#                 print ' already got lyrics for artist, track_id = ', track_id_selected
#             if not track_id_selected in df_mxm_info.track_id.values:
#                 # Get Lyrics!
#                 try:
#                     lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
#                     # Append lyrics to list
#                     lyrics_list.append(lyrics.values())
#                     # subset[is_true] is info for the best match to the billboard title+artist 
#                     is_true = subset['track_id'] == track_id_selected     
#                     add_to_mxm_info = subset[is_true]
#                     #add col with hot100 artist name, and one with hot100 song name
#                     add_to_mxm_info['hot100 artist'] = artist_hot
#                     add_to_mxm_info['hot100 track'] = trackname_hot
#                     add_to_mxm_info['artist string match score'] = result['score']
#                     #also add hot100 spotify_id
#                     add_to_mxm_info['hot100 SpotifyID'] = spotify_hot
#                     # append new song info to df_mxm_info
#                     df_mxm_info = df_mxm_info.append(add_to_mxm_info)
#                     #check that lyrics list is same length as info df
#                     # assert (len(lyrics_list) == len(df_mxm_info))
#                     df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols)
#                     # merge these 2 dataframes
#                     df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')
#                 except : 
#                     print 'no lyrics for track id', track_id_selected
#                     hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#                     hot_tracknames_no_lyrics.append(trackname_hot) 
#                     pass
#         else: # try the reverse way
#             #ask mxm for info about this track (search on trackname first)
#             collection2 = TracksCollection.fromSearch(q_artist=artist_hot, apikey = apikey)
#             if len(collection2)==0:
#                 print('No lyrics found')
#                 hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#                 hot_tracknames_no_lyrics.append(trackname_hot) 
#                 continue # skip this row of df_reduced
#             #store collection in dataframe called 'subset'
#             subset2 = pd.DataFrame.from_dict(collection2)[cols]
#             #filter for only songs with lyrics:
#             subset2 = subset2[subset2['has_lyrics']==1]
#             artists2 = subset2['artist_name'].values
#             tracks2 = subset2['track_name'].values
#             sids2 = subset2['track_spotify_id'].values
#             track_id2 = subset2['track_id'].values
#             if len(tracks2)==0:
#                 print('No lyrics found')
#                 hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#                 hot_tracknames_no_lyrics.append(trackname_hot) 
#                 continue # skip this row of df_reduced
#             print 'There are ', len(tracks2), ' tracks from this artist'
#             # fuzzy match the artist from hot100 to all artists matching track name
#             try:
#                 result = find_artist_match(trackname_hot, tracks)
#             except AssertionError:
#                 continue
#             ind2 = result['ind']
#             score2 = result['score']
#             #print 'matched track name = ', tracks[ind]
#             if score2 > score or sids2[ind2]==spotify_hot:
#                 print 'matched track name = ', tracks2[ind2]
#                 # select the best track id (best match to artist name)
#                 track_id_selected = track_id2[ind2]
#                 # now get the lyrics, but only if the 'track_id' is not already in the info df
#                 if track_id_selected in df_mxm_info.track_id.values:
#                     print ' already got lyrics for artist, track_id = ', track_id_selected
#                 if not track_id_selected in df_mxm_info.track_id.values:
#                     # Get Lyrics!
#                     try:
#                         lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
#                         # Append lyrics to list
#                         lyrics_list.append(lyrics.values())
#                         # subset[is_true] is info for the best match to the billboard title+artist 
#                         is_true = subset2['track_id'] == track_id_selected     
#                         add_to_mxm_info = subset2[is_true]
#                         #add col with hot100 artist name, and one with hot100 song name
#                         add_to_mxm_info['hot100 artist'] = artist_hot
#                         add_to_mxm_info['hot100 track'] = trackname_hot
#                         add_to_mxm_info['track string match score'] = score2
#                         #also add hot100 spotify_id
#                         add_to_mxm_info['hot100 SpotifyID'] = spotify_hot
#                         # append new song info to df_mxm_info
#                         df_mxm_info = df_mxm_info.append(add_to_mxm_info)
#                         #check that lyrics list is same length as info df
#                         # assert (len(lyrics_list) == len(df_mxm_info))
#                         df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols)
#                         # merge these 2 dataframes
#                         df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')
#                     except : 
#                         print 'no lyrics for track id', track_id_selected
#                         hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#                         hot_tracknames_no_lyrics.append(trackname_hot) 
#                         pass
#             else: #use the previous search result
#                 print 'matched artist name = ', artists[ind]
#                 # select the best track id (best match to artist name)
#                 track_id_selected = track_id[ind]
#                 # now get the lyrics, but only if the 'track_id' is not already in the info df
#                 if track_id_selected in df_mxm_info.track_id.values:
#                     print ' already got lyrics for artist, track_id = ', track_id_selected
#                 if not track_id_selected in df_mxm_info.track_id.values:
#                     # Get Lyrics!
#                     try:
#                         lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
#                         # Append lyrics to list
#                         lyrics_list.append(lyrics.values())
#                         # subset[is_true] is info for the best match to the billboard title+artist 
#                         is_true = subset['track_id'] == track_id_selected     
#                         add_to_mxm_info = subset[is_true]
#                         #add col with hot100 artist name, and one with hot100 song name
#                         add_to_mxm_info['hot100 artist'] = artist_hot
#                         add_to_mxm_info['hot100 track'] = trackname_hot
#                         add_to_mxm_info['artist string match score'] = result['score']
#                         #also add hot100 spotify_id
#                         add_to_mxm_info['hot100 SpotifyID'] = spotify_hot
#                         # append new song info to df_mxm_info
#                         df_mxm_info = df_mxm_info.append(add_to_mxm_info)
#                         #check that lyrics list is same length as info df
#                         # assert (len(lyrics_list) == len(df_mxm_info))
#                         df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols)
#                         # merge these 2 dataframes
#                         df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')
#                     except : 
#                         print 'no lyrics for track id', track_id_selected
#                         hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
#                         hot_tracknames_no_lyrics.append(trackname_hot) 
#                         pass
#     # Save the data frame!
#     df_mxm.to_pickle(name2save)
#     # Also save lists of no match:
#     if len(hot_artists_no_lyrics)>0:
#         #save monthly datelist to csv:
#         myfile = open('NoLyricsList_'+ str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)+'.csv', 'wb')
#         wr = csv.writer(myfile)
#         wr.writerow('Artists with no lyrics found:')
#         wr.writerow(hot_artists_no_lyrics)
#         wr.writerow('Corresponding track names:')
#         wr.writerow(hot_tracknames_no_lyrics)

In [42]:
no_match = {'artists': hot_artists_no_lyrics, 'tracks': hot_tracknames_no_lyrics}
no_match_df = pd.DataFrame(data=no_match)
print '# no match before de-dup:', len(no_match_df)
print '# no match after de-dup:', len(no_match_df.drop_duplicates())
no_match_df.drop_duplicates()

# no match before de-dup: 26
# no match after de-dup: 16


,artists,tracks
0,The Chipmunks With David Seville,The Chipmunk Song
1,Billy Grammer,Gotta Travel On
2,Reg Owen & His Orchestra,Manhattan Spiritual
5,Annette With The Afterbeats,Tall Paul
6,Chris Barber's Jazz Band,Petite Fleur (Little Flower)
7,Cyril Stapleton And His Orchestra,The Children's Marching Song (Nick Nack Taddy ...
8,Thomas Wayne With the Delons,Tragedy
12,Tommy Dee with Carol Kay and the Teen-Aires,Three Stars
13,The Exotic Sounds of Martin Denny,Quiet Village
14,Franck Pourcel's French Fiddles,Only You


In [14]:
#for y in xrange(2007,2016): (for finishing the 2000s)
for y in xrange(2014,2017): #get the 70s!

    ## Initialize data frames and lists:
    apikey = mxm_key

    #header for info to be stored in df_mxm_info
    cols = ['lyrics_id', 'has_lyrics', 'artist_name', 'track_spotify_id','track_id','track_name']

    #header for info in df_mxm_lyrics
    lyrics_df_cols = ['lyrics_copyright', 'lyrics_body','publisher_list', 'can_edit', 'writer_list', \
                      'instrumental','verified','html_tracking_url','lyrics_language_description', \
                      'locked','restricted','explicit','updated_time','lyrics_id','script_tracking_url', \
                      'action_requested','pixel_tracking_url','lyrics_language']

    # initialize dfs and lists :
    #if 'df_mxm_info' not in locals():
    df_mxm_info = pd.DataFrame(columns = cols) #dataframe for some song info (from collections object)
    #if 'df_mxm_lyrics' not in locals():
    df_mxm_lyrics = pd.DataFrame(columns = lyrics_df_cols) #dataframe for lyrics and related attributes
    #if 'lyrics_list' not in locals():
    lyrics_list = [] #append lyric info to a list for each song, later we'll put it in a dataframe!
    #if 'hot_artists_no_lyric' not in locals():
    hot_artists_no_lyrics = [] #when we don't find lyrics, store artistname here
    #if 'hot_tracknames_no_lyrics' not in locals():
    hot_tracknames_no_lyrics = [] #when we don't find lyrics, store trackname here
    print 'df_mxm_info has ', len(df_mxm_info), ' rows' 
    print 'df_mxm_lyrics has ', len(df_mxm_info), ' rows' 
    print 'lyrics_list has length ', len(df_mxm_info)

    #subset df_monthly given start_date, end_date, and max_rank:
    start_date = datetime.date(year=y,month=1,day=1)
    end_date = datetime.date(year=y+1,month=1,day=1)
    max_rank = 100 #just top N songs

    df_reduced = df[(df['date'] > start_date) & (df['date']< end_date) & (df['Rank']<=max_rank)]

    print ' number songs in df_reduced = ', len(df_reduced)

    #name for saving df
    name2save = './Top100/Lyrics_'+str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)
    print 'will save df as : ', name2save

    #now get the lyrics
    for i in xrange(len(df_reduced)):
        trackname_hot = df_reduced['Title'].iloc[i]
        #fix unicode error neil diamond desire song...
        if trackname_hot=='Desir\xc3\x83\xc2\x83\xc3\x82\xc2\xa9e':
            trackname_hot = 'Desire'
        #fix for san jose
        if trackname_hot=='Do You Know The Way To San Jos\xc3\x83\xc2\x83\xc3\x82\xc2\xa9':
            trackname_hot = 'Do You Know The Way To San Jose'
        if trackname_hot=='Nel Blu Dipinto Di Blu (Volar\xc3\x83\xc2\x83\xc3\x82\xc2\xa9)':
            trackname_hot = 'Nel Blu Dipinto Di Blu Volar'
        if trackname_hot=='\xc3\x83\xc2\x82\xc3\x82\xc2\xbfD\xc3\x83\xc2\x83\xc3\x82\xc2\xb2nde Est\xc3\x83\xc2\x83\xc3\x82\xc2\xa0 Santa Claus? (Where Is Santa Claus?)':
            trackname_hot = 'Where Is Santa Claus?'
        artist_hot = df_reduced['Artist'].iloc[i]
        spotify_hot = df_reduced['SpotifyID'].iloc[i]
        print 'processing ', trackname_hot, ' by ', artist_hot

        #ask mxm for info about this track (search on trackname first)
        try:
            collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)
        except UnicodeDecodeError:
            # search thingy couldn't read this song, add to list
            print('unicode error')
            hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
            hot_tracknames_no_lyrics.append(trackname_hot) 
            continue # skip this row of df_reduced
        if len(collection)==0:
            print('No lyrics found')
            hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
            hot_tracknames_no_lyrics.append(trackname_hot) 
            continue # skip this row of df_reduced
        #store collection in dataframe called 'subset'
        subset = pd.DataFrame.from_dict(collection)[cols]
        #filter for only songs with lyrics:
        subset = subset[subset['has_lyrics']==1]
        #move on if no matches found...
        if len(subset)==0:
            print ' No lyrics found ...'
            hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
            hot_tracknames_no_lyrics.append(trackname_hot) 
            continue # skip this row of df_reduced -- in future we should try other searches when this initial search fails
        artists = subset['artist_name'].values
        tracks = subset['track_name'].values
        sids = subset['track_spotify_id'].values
        track_id = subset['track_id'].values
        print 'There are ', len(artists), ' artists with this song title'
        # fuzzy match the artist from hot100 to all artists matching track name
        try:
            result = find_artist_match(artist_hot, artists)
        except AssertionError:
            continue
        ind = result['ind']
        score = result['score']
        #if score >= 75 or spotify_hot==sids[ind]:
        print 'matched artist name = ', artists[ind]
        # select the best track id (best match to artist name)
        track_id_selected = track_id[ind]
        #spot_id_selected = sids[ind]
        # now get the lyrics, but only if the 'track_id' is not already in the info df
        if track_id_selected in df_mxm_info.track_id.values:
            print ' already got lyrics for artist, track_id = ', track_id_selected
        if not track_id_selected in df_mxm_info.track_id.values:
            # Get Lyrics!
            try:
                lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
                # Append lyrics to list
                lyrics_list.append(lyrics.values())
                # subset[is_true] is info for the best match to the billboard title+artist 
                is_true = subset['track_id'] == track_id_selected     
                add_to_mxm_info = subset[is_true]
                #add col with hot100 artist name, and one with hot100 song name
                add_to_mxm_info['hot100 artist'] = artist_hot
                add_to_mxm_info['hot100 track'] = trackname_hot
                add_to_mxm_info['artist string match score'] = result['score']
                #also add hot100 spotify_id
                add_to_mxm_info['hot100 SpotifyID'] = spotify_hot
                # append new song info to df_mxm_info
                df_mxm_info = df_mxm_info.append(add_to_mxm_info)
                #check that lyrics list is same length as info df
                # assert (len(lyrics_list) == len(df_mxm_info))
                df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols)
                # merge these 2 dataframes
                df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')
            except : 
                print 'no lyrics for track id', track_id_selected
                hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
                hot_tracknames_no_lyrics.append(trackname_hot) 
                pass
       
    # Save the data frame!
    df_mxm.to_pickle(name2save)
    # Also save lists of no match:
    if len(hot_artists_no_lyrics)>0:
        #save monthly datelist to csv:
        myfile = open('./Top100NoMatch/NoLyricsList_'+ str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)+'.csv', 'wb')
        wr = csv.writer(myfile)
        wr.writerow('Artists with no lyrics found:')
        wr.writerow(hot_artists_no_lyrics)
        wr.writerow('Corresponding track names:')
        wr.writerow(hot_tracknames_no_lyrics)

df_mxm_info has  0  rows
df_mxm_lyrics has  0  rows
lyrics_list has length  0
 number songs in df_reduced =  1300
will save df as :  ./Top100/Lyrics_2014-01-01-2015-01-01MaxRank100
processing  The Monster  by  Eminem Featuring Rihanna
There are 

/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

 8  artists with this song title
Best Match =  Eminem
Score =  40
matched artist name =  Eminem
processing  Timber  by  Pitbull Featuring Ke$ha
There are  10  artists with this song title
Best Match =  PitBull feat. Ke$ha
Score =  76
matched artist name =  PitBull feat. Ke$ha
processing  Counting Stars  by  OneRepublic
There are  10  artists with this song title
Best Match =  OneRepublic
Score =  100
matched artist name =  OneRepublic
processing  Royals  by  Lorde
There are  9  artists with this song title
Best Match =  Lounge Cafe
Score =  38
matched artist name =  Lounge Cafe
processing  Say Something  by  A Great Big World & Christina Aguilera
There are  10  artists with this song title
Best Match =  Christina Aguilera
Score =  64
matched artist name =  Christina Aguilera
processing  Demons  by  Imagine Dragons
There are  10  artists with this song title
Best Match =  Imagine Dragons
Score =  100
matched artist name =  Imagine Dragons
processing  Let Her Go  by  Passenger
There are 

/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
no_match = {'artists': hot_artists_no_lyrics, 'tracks': hot_tracknames_no_lyrics}
no_match_df = pd.DataFrame(data=no_match)
print '# no match before de-dup:', len(no_match_df)
print '# no match after de-dup:', len(no_match_df.drop_duplicates())
no_match_df.drop_duplicates()

# no match before de-dup: 69
# no match after de-dup: 42


,artists,tracks
0,The Archies,Feelin' So Good (S.k.o.o.b.y-D.o.o)
1,O.C. Smith,Isn't It Lonely Together
3,The Meters,Sophisticated Cissy
4,Rock & Roll Double Bubble,Bubble Gum Music
5,The T.S.U. Toronadoes,Getting The Corners
7,Rhinoceros,Apricot Brandy
8,Shango,Day After Day (It's Slippin' Away)
9,The Ventures,Hawaii Five-O
10,Booker T. & The MG's,Time Is Tight
14,James Brown,I Don't Want Nobody To Give Me Nothing (Open U...


In [51]:
if len(hot_artists_no_lyrics)>0:
    #save monthly datelist to csv:
    myfile = open('./Top100NoMatch/NoLyricsList_'+ str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)+'.csv', 'wb')
    wr = csv.writer(myfile)
    wr.writerow('Artists with no lyrics found:')
    wr.writerow(hot_artists_no_lyrics)
    wr.writerow('Corresponding track names:')
    wr.writerow(hot_tracknames_no_lyrics)

### These next few cells are just checking the most recent data frame to see how it looks.

In [17]:
name2save

'./Lyrics_1978-01-01-1979-01-01MaxRank20'

In [36]:
df[df['date']<datetime.date(year=1960,month=1,day=1)]

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09
5,Duane Eddy His Twangy Guitar And The Rebels,Re-Entry,0,6,6,6gIFUpY85r9b4pCDIIkJDF,https://embed.spotify.com/?uri=spotify:track:6...,Rebel-'Rouser,8,1958-08-09
6,The Coasters,Re-Entry,0,7,7,3h5g8M5cWCPniE29PHhv7M,https://embed.spotify.com/?uri=spotify:track:3...,Yakety Yak,7,1958-08-09
7,Jack Scott,Re-Entry,0,3,8,2urRqmAFhjZKo8Z6sEGzEv,https://embed.spotify.com/?uri=spotify:track:2...,My True Love,14,1958-08-09
8,The Johnny Otis Show,Re-Entry,0,9,9,5gjeRYKyOH23oNNb59WjJq,https://embed.spotify.com/?uri=spotify:track:5...,Willie And The Hand Jive,10,1958-08-09
9,Peggy Lee,Re-Entry,0,8,10,2CeqxyOZEyiL6pTDYZ9gPH,https://embed.spotify.com/?uri=spotify:track:2...,Fever,12,1958-08-09


In [23]:
cols_to_view = ['artist string match score','artist_name','hot100 artist','track_name','lyrics_body']
df_mxm[cols_to_view].tail()

,artist string match score,artist_name,hot100 artist,track_name,lyrics_body
28,44.0,Alabama feat. 'N Sync,N Sync,God Must Have Spent a Little More Time On You,Can this be true? Tell me can this be real?\nH...
29,33.0,Go Go Berlin,Jesse Powell,You You You,"Yeah this one thing, that I need\nIs for you t..."
30,100.0,Case,Case,Faded Pictures (feat. Joe),"That's right\nLife is funny, ain't it?\nSometi..."
31,100.0,Gerald Levert,Gerald Levert,Taking Everything,"To my heart\nTo my car\nTo my crib\nMy kids, e..."
32,38.0,Labbayk,Sugar Ray,Every Morning,Home > Labbayk > Every Morning\n\nEvery mornin...


In [24]:
df_mxm[df_mxm['artist_name']=='Nickelback']
df_mxm[df_mxm['track_id']==82751560][['lyrics_body']].values

array([], shape=(0, 1), dtype=object)

In [25]:
len(df_mxm)

33

In [26]:
df_mxm['lyrics_body'].values[0:10]

array([ u"No mountain's too high for you to climb\nAll you have to do is have some climbing faith, oh yeah\nNo river's too wide for you to make it across\nAll you have to do is believe it when you pray\nAnd then you will see the morning will come\nAnd every day will be bright as the sun\nAll of your fears cast them on me\nI just want you to see\n\nI'll be your cloud up in the sky\nI'll be your shoulder when you cry\nI hear your voices when you call me\nI am your angel\nAnd when all hope is gone, I'm here\nNo matter how far you are, I'm near\nIt makes no difference who you are\nI am your angel\n\nI'm your angel\n\nI saw your teardrops and I heard you cry\nAll you need is time seek me and you shall find\nYou have everything and you're still lonely\nIt don't have to be this way\nLet me show you a better day\n...\n\n******* This Lyrics is NOT for Commercial use *******",
       u'How did you get here? Nobody\'s supposed to be here\nI\'ve tried that love thing for the last time\nMy heart sa

## note: the fuzzy matching can sometimes pull the wrong lyrics, and it's not clear what cutoff string matching score should be, for example:
### Low score but true match:
processing  This I Promise You  by  'N Sync

There are  10  artists with this song title

Best Match =  \*NSYNC

Score =  31 
### Low score, false match:
processing  I Wish  by  R. Kelly

There are  10  artists with this song title

Best Match =  One Direction

Score =  19

### Higher score, false match:
processing  Stronger  by  Britney Spears
There are  9  artists with this song title
Best Match =  Kanye West
Score =  42
matched artist name =  Kanye West

### Should think more about this later, but for now I'm not cutting anything off and instead have a column in lyrics section for artist and trackname of the hot100 track used to find these lyrics
